In [176]:
import pandas , numpy
features = pandas.read_csv('./features.csv', index_col='match_id')
test_features = pandas.read_csv('./features_test.csv', index_col='match_id')

In [177]:
#empty cells count:
for column in features:
    dif = len( features.index ) - features[ column ].count()
    if dif > 0:
        print "'" + column + "'"

'first_blood_time'
'first_blood_team'
'first_blood_player1'
'first_blood_player2'
'radiant_bottle_time'
'radiant_courier_time'
'radiant_flying_courier_time'
'radiant_first_ward_time'
'dire_bottle_time'
'dire_courier_time'
'dire_flying_courier_time'
'dire_first_ward_time'


'first_blood_time' empty cells :  19553
'first_blood_team' empty cells :  19553
'first_blood_player1' empty cells :  19553
'first_blood_player2' empty cells :  43987
'radiant_bottle_time' empty cells :  15691
'radiant_courier_time' empty cells :  692
'radiant_flying_courier_time' empty cells :  27479
'radiant_first_ward_time' empty cells :  1836
'dire_bottle_time' empty cells :  16143
'dire_courier_time' empty cells :  676
'dire_flying_courier_time' empty cells :  26098
'dire_first_ward_time' empty cells :  1826

*_first_ward_time пропуски потому что не было вардов в игре. Тоже самое с first_blood, в игре не было убийств. Если с количеством предметов в игре можно обойтись "0" в случае, если таковых приобретено в течении игры не было, то в случае с характеристиками предметов, коих приобретено не было, все несколько сложнее, поэтому и пропуск.

In [178]:
features.replace( '' , numpy.nan , inplace=True )
features.fillna( 0,inplace=True)
test_features.replace( '' , numpy.nan , inplace=True )
test_features.fillna(0, inplace=True)

целевая переменная : "radiant_win"

In [179]:
#разделим данные на матрицу характеристик и вектор классов
ignore_labels = ["radiant_win" , "match_id"]
X = features.select( lambda x: x in test_features.columns.values and not x in ignore_labels , axis=1 ).as_matrix()
Y = features.select( lambda x: x == "radiant_win" , axis=1 ).as_matrix().ravel()

In [184]:
from sklearn.svm.libsvm import predict_proba
from sklearn.ensemble import GradientBoostingClassifier
import random
from sklearn.metrics import log_loss , roc_auc_score
def testAbst(cv,N,X,Y):
    #pick random samples from indices vector to speed up calculation
    indices = random.sample(range(0,len(X)), N)
    x = X[ indices ]
    y = Y[ indices ]
    from sklearn.model_selection import KFold , cross_val_score
    kf = KFold( n_splits = 5 , shuffle = True , random_state=241 )
    sum = 0.0
    for train_indices, test_indices in kf.split(x):
        cv.fit( x[train_indices] , y[train_indices] )
        pred = cv.predict_proba(x[test_indices])[:, 1]
        sum+= roc_auc_score( y[test_indices] , pred )
    #return avg area under roc
    return sum/5
#5000 is enough
def testBoosting(range,N,X,Y):
    print "Boosting"
    for num in range:
        print num, ":" , testAbst(GradientBoostingClassifier(n_estimators=num, random_state=241),N,X,Y)
def testLogReg(range,N,X,Y):
    print "logReg"
    for regk in range:
        print regk , ":" , testAbst(LogisticRegression(penalty ="l2",C=regk),N,X,Y)

In [185]:
import time
import datetime

start_time = datetime.datetime.now()

testBoosting([ 30 ],X.shape[0],X,Y)

print 'Time elapsed:', datetime.datetime.now() - start_time

Boosting
30 : 0.689744877956
Time elapsed: 0:01:10.076000


In [186]:
testBoosting([ 10 , 20 , 30 , 100 ],5000,X,Y)

Boosting
10 : 0.658525714926
20 : 0.666494971528
30 : 0.671254649462
100 : 0.7042305657


результат улучшается, но время обучения тоже. Для ускорения можно использовать подмножество выборки.

In [188]:
start_time = datetime.datetime.now()

testLogReg([ 1.0e-3 ],X.shape[0],X,Y)

print 'Time elapsed:', datetime.datetime.now() - start_time
testLogReg([ 1.0e-3 , 1.0e-2 , 1.0e-1 , 1.0e1 , 1.0e2 ],5000,X,Y)

logReg
0.001 : 0.513446681145
Time elapsed: 0:00:01.679000
logReg
0.001 : 0.519001063905
0.01 : 0.505890125162
0.1 : 0.51830330524
10.0 : 0.519764958233
100.0 : 0.517493812543


качество логистической регрессии ниже, в виду наличия шума в признаках. время обучения ниже.

In [189]:
ignore_labels = ["radiant_win" , "match_id","lobby_type","start_time","r1_hero","r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"]
X = features.select( lambda x: x in test_features.columns.values and not x in ignore_labels , axis=1 ).as_matrix()
Y = features.select( lambda x: x == "radiant_win" , axis=1 ).as_matrix().ravel()
testLogReg([ 1.0e-3 , 1.0e-2 , 1.0e-1 , 1.0e1 , 1.0e2 ],5000,X,Y)
testBoosting([ 10 , 20 , 30 , 100 ],5000,X,Y)


logReg
0.001 : 0.711083866188
0.01 : 0.712162982448
0.1 : 0.705755304459
10.0 : 0.697817089818
100.0 : 0.70474231211
Boosting
10 : 0.651068097338
20 : 0.668242749842
30 : 0.674395240321
100 : 0.682008725538


Как и ожидалось, качество классификатора основанного на решающих деревьях не изменилось, в то время как качество линейного классификатора значительно повысилось.

In [190]:
category_labels = ["r1_hero","r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"]
cathegory_dataframe = features.select( lambda x: x in test_features.columns.values and x in category_labels , axis=1 )
unique_heroes = set()
max_hero_id = 0
for column in cathegory_dataframe:
    values = cathegory_dataframe[column].value_counts()
    unique_heroes |= set(values.index)
unique_heroes = list(unique_heroes)
unique_heroes.sort()
max_hero_id = unique_heroes[ len(unique_heroes) - 1 ]
print "max hero id:" , max_hero_id
print "unique heroes count in sample :", len(unique_heroes)

max hero id: 112
unique heroes count in sample : 108


In [191]:
pick_matrix = numpy.zeros(shape=(len(cathegory_dataframe.index),max_hero_id))
for i, match_id in enumerate(cathegory_dataframe.index):
    for p in xrange(5):
        pick_matrix[i, cathegory_dataframe.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        pick_matrix[i, cathegory_dataframe.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [192]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_full = numpy.concatenate([X,pick_matrix],axis=1)

In [193]:
testLogReg([ 1.0e-3 , 1.0e-2 , 1.0e-1 , 1.0e1 , 1.0e2 ],5000,X_full,Y)

logReg
0.001 : 0.718879007733
0.01 : 0.730229437803
0.1 : 0.721349027288
10.0 : 0.742075073804
100.0 : 0.726907938081


качество классификатора после учета героев повысилась.

In [194]:
samples = random.sample(range(0,len(X)), 5000)
logcv = LogisticRegression(penalty ="l2",C=0.01)
logcv.fit( X_full[samples] , Y[samples] )
Y_predict = logcv.predict_proba( X_full )
print "maximum and minimum probabilities:" , Y_predict.max() , Y_predict.min()

maximum and minimum probabilities: 0.998852190469 0.00114780953128
